# Naïve Algorithm

The Naïve Algorithm is a fundamental approach to solving the maximum s-t (source-to-sink) flow problem in network flow analysis. This algorithm provides an elementary method for determining the maximum flow that can be sent from a source node $s$ to a sink node $t$ within a flow network. The process involves initially sending flow along any path from $s$ to $t$ that has available capacity. The key principle behind the Naïve Algorithm is to repeatedly find such paths and push as much flow as possible along them until no further augmenting paths exist.

Here we use example in Figure 3.1: An input for the max s-t flow problem.

<img src="maxFlowQ1.jpeg" alt="" width="400"/>

We could transform this graph to a table as follows:

$$
\begin{matrix}
   & S & A & B & C & D & E & T \\
 S & 0 & 3 & 1 & 1 & 0 & 0 & 0 \\
 A & 0 & 0 & 1 & 0 & 1 & 0 & 0 \\
 B & 0 & 1 & 0 & 0 & 0 & 3 & 0 \\
 C & 0 & 0 & 0 & 0 & 4 & 2 & 0 \\
 D & 0 & 0 & 0 & 0 & 0 & 0 & 4 \\
 E & 0 & 0 & 0 & 4 & 0 & 0 & 1 \\
 T & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{matrix}
$$

and hence generate a capacity matrix Q:

$$
Q=\left[
\begin{matrix}
 0 & 3 & 1 & 1 & 0 & 0 & 0 \\
 0 & 0 & 1 & 0 & 1 & 0 & 0 \\
 0 & 1 & 0 & 0 & 0 & 3 & 0 \\
 0 & 0 & 0 & 0 & 4 & 2 & 0 \\
 0 & 0 & 0 & 0 & 0 & 0 & 4 \\
 0 & 0 & 0 & 4 & 0 & 0 & 1 \\
 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{matrix}
\right]
$$

**Step 1:** Find a shortest path from $s$ to $t$ without considering the weight of each edge.

In [10]:
from collections import deque

def bfs_path(graph, start, goal):
    queue = deque([(start, [start])])  # queue of (current node, path to current node)
    visited = set()
    visited.add(start)
    
    while queue:
        current, path = queue.popleft()
        if current == goal:
            return path
        for neighbor in range(len(graph[current])):
            if graph[current][neighbor] != 0 and neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    return None  # return None if no path is found

**Step 2:** Calculates the minimum capacity along the path and then adjusts the residual of the edges accordingly.

In [11]:
def update_residual_graph(graph, path):
    min_capacity = float('inf')
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        if graph[start][end] < min_capacity:
            min_capacity = graph[start][end]
    
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        graph[start][end] -= min_capacity
    return min_capacity


** 

In [12]:
def main():
    # Example input matrix Q
    Q = [[0, 3, 1, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 3, 0],
        [0, 0, 0, 0, 4, 2, 0],
        [0, 0, 0, 0, 0, 0, 4],
        [0, 0, 0, 4, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]]

    total_flow = 0
    source = 0
    sink = 6

    while True:
        path = bfs_path(Q, source, sink)
        if path is None:
            break
        flow = update_residual_graph(Q, path)
        total_flow += flow

    print("Maximum flow:", total_flow)
    print("Updated Residual Graph:")
    for row in Q:
        print(row)

if __name__ == '__main__':
    main()

Maximum flow: 4
Updated Residual Graph:
[0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 2, 2, 0]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 3, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]


Although this algorithm serves as a conceptual tool for understanding the max flow problem, it is generally not efficient for large networks due to its lack of optimization mechanisms, such as those found in more advanced algorithms like the Ford-Fulkerson method or the Edmonds-Karp algorithm. Nonetheless, studying the Naïve Algorithm provides valuable insights into the basic mechanics of flow algorithms and the foundation upon which more sophisticated techniques are built.

The problem can then be naturally set up as an LP by using a variable for the flow along each edge.

$$
\begin{alignat*}{3}
&&\max \sum_{v:(s,v)\in E}f_{(s,v)}&&-\sum_{v:(s,v)\in E}f_{(v,s)} &&\\
&& s.t.&&\forall(u,v)\in E && f_{(u,v)}\geq0 \\
&& &&\forall(u,v)\in E && f_{(u,v)}\leq C_{(u,v)} 
\end{alignat*}
$$

$$
\forall v \neq s,t \quad \sum_{u:(u,v)\in E}f_{(u,v)}=\sum_{w:(v,w)}\in E f_{(v,w)}
$$